In [2]:
import sys
import os

from dotenv import load_dotenv

from agents import Agent

sys.path.append(os.path.join(os.path.dirname(os.getcwd()), "src"))

from mav.MAS.pipeline import Pipeline

from mav.benchmark import benchmark_suite_without_injections

from mav.Tasks.load_task_suites import get_suite
from mav.Tasks.utils._transform import convert_to_openai_function_tool

from mav.MAS.framework import MultiAgentSystem

load_dotenv()

True

In [3]:
# loading a task suite
slack_task_suite = get_suite("slack")
banking_task_suite = get_suite("banking")
travel_task_suite = get_suite("travel")

# convert to openai function tools
banking_openai_tools = []
slack_openai_tools = []
travel_openai_tools = []

for tool in travel_task_suite.tools:
    try:
        travel_openai_tools.append(convert_to_openai_function_tool(tool))
    except Exception as e:
        print(f"Error converting tool {tool.__name__}: {e}")

for tool in banking_task_suite.tools:
    try:
        banking_openai_tools.append(convert_to_openai_function_tool(tool))
    except Exception as e:
        print(f"Error converting tool {tool.__name__}: {e}")

for tool in slack_task_suite.tools:
    try:
        slack_openai_tools.append(convert_to_openai_function_tool(tool))
    except Exception as e:
        print(f"Error converting tool {tool.__name__}: {e}")

In [7]:
banking_agent = Agent(
    name="Banking Agent",
    instructions="""You are an intelligent banking agent that handles user queries using available tools.""",
    model="gpt-4o",
    tools=banking_openai_tools,
)

slack_agent = Agent(
    name="Slack Agent",
    instructions="""You are a Slack agent that interacts with users on Slack and handles their requests using available tools.""",
    model="gpt-4o",
    tools=slack_openai_tools,
)

travel_agent = Agent(
    name="Travel Agent",
    instructions="""You are a travel agent that assists users with their travel plans and bookings using available tools.""",
    model="gpt-4o",
    tools=travel_openai_tools,
)

# Sequential Workflow

In [ ]:
summary_agent = Agent(
    name="Summary Agent",
    instructions="""You are an intelligent response summarization agent that summarizes the responses from other agents.""",
    model="gpt-4o",
)

banking_agent.handoffs = [summary_agent]

mas = MultiAgentSystem(
    mas=banking_agent
)

# create a pipeline
pipeline = Pipeline(
    mas=mas
)

results = await benchmark_suite_without_injections(
    agent_pipeline=pipeline,
    suite=banking_task_suite,
)

[non-fatal] Tracing: server error 500, retrying.
[non-fatal] Tracing: server error 500, retrying.
[non-fatal] Tracing: server error 500, retrying.
[non-fatal] Tracing: server error 500, retrying.
[non-fatal] Tracing: server error 500, retrying.
[non-fatal] Tracing: server error 500, retrying.
[non-fatal] Tracing: server error 500, retrying.
[non-fatal] Tracing: server error 500, retrying.
[non-fatal] Tracing: server error 500, retrying.
[non-fatal] Tracing: server error 500, retrying.


[non-fatal] Tracing: server error 500, retrying.


# Handoffs Workflow

In [ ]:
triage_agent = Agent(
    name="Manager Agent",
    instructions="""You are a manager agent that triages tasks to other agents based on their expertise.""",                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
    model="gpt-4o",
    handoffs=[banking_agent, slack_agent, travel_agent],
)

mas = MultiAgentSystem(
    mas=banking_agent
)

# create a pipeline
pipeline = Pipeline(
    mas=mas
)

results = await benchmark_suite_without_injections(
    agent_pipeline=pipeline,
    suite=banking_task_suite,
)

# Agents as Tools

In [ ]:
orchestrator_agent = Agent(
    name="orchestrator_agent",
    instructions=(
        "You are an orchestrator agent that uses different tools to handle user requests. "
    ),
    tools=[
        banking_agent.as_tool(
            tool_name="banking_tool",
            tool_description="This tool handles banking-related queries and tasks.",
        ),
        slack_agent.as_tool(
            tool_name="slack_tool",
            tool_description="This tool interacts with users on Slack and handles their requests.",
        ),
        travel_agent.as_tool(
            tool_name="travel_tool",
            tool_description="This tool assists users with their travel plans and bookings.",
        ),
    ],
)

mas = MultiAgentSystem(
    mas=banking_agent
)

# create a pipeline
pipeline = Pipeline(
    mas=mas
)

results = await benchmark_suite_without_injections(
    agent_pipeline=pipeline,
    suite=banking_task_suite,
)